In [1]:
# default_exp core

# module name here

> API details.

In [2]:
#hide 
#!pip install nbdev marko

In [3]:
#hide
from nbdev.showdoc import *

In [4]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

[nb_2_html](https://github.com/BNIA/dataplay/blob/main/notebooks/04_nb_2_html.ipynb)

In [5]:
#export
import re
from IPython.display import clear_output
import marko

In [6]:
#export
from nbdev import export2html
from nbdev.export2html import read_nb, get_metadata, clean_exports
from nbdev.export import split_flags_and_code, check_re_multi

In [7]:
#export
# 1st fn. passes each cell to decision fn.
def convertNb(nb):  return [cleanCell(c) for c in nb['cells']]

# 2nd fn. returns text or passes cell to 'code cell' processor
def cleanCell(cell): return marko.convert(cell.source) if cell.cell_type == 'markdown' else processCode(cell)

# 3rd fn. Calls getFlags, processSource, processOutput
def processCode(cell):
  x = [cell.source]
  flags = getFlags(x[0])
  x[0] = processSource( x[0], flags )
  if ( len(cell.outputs) ):
    for o in cell.outputs: x.append( processOutput(o, flags) )
  return x

In [8]:
#export
# Detect and stripout and handle flags.
def getFlags(source):
  input_aug = ['#collapse_input_open', '#collapse_input', '#collapse_output_open', '#collapse_output',
               '#hide_input', '#hide_output', '#hide ', '%%capture', '%%javascript', '%%html']
  return [ x for x in input_aug if re.search(x, source) ]

# enclose the text within a collapsable html drawer.
def makeDetails( content, open ):
  return "<details "+('open' if open else '')+"> <summary>Click to toggle</summary> "+content+"</details>"

In [9]:
#export
# Strip Flags from text, make details, hide all.
def processSource (source, flags):
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_input_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_input'): source = makeDetails( source, False )
    if (lbl == '#hide '): source = ''
    if (lbl == '#hide_input'): source = ''
    if (lbl == '%%javascript'): source = ''
    if (lbl == '%%html'): source = ''
    if (lbl == '%%capture'): source = ''
  return source

In [10]:
#export
def processOutput(source, flags):
  if(source.output_type=='error'):  return ''
  if(source.output_type=='stream'):
    if(source.name=='stderr'): return ''
    source.data = {'text/html': source.text} # This will have the stream process as text/html.
  # print(source.data.keys())
  if ( hasattr(source.data, 'text/html') ): source=source.data['text/html'];
  elif ( hasattr(source.data, 'application/javascript') ): source='<script>'+source.data['application/javascript']+'</script>'
  elif ( hasattr(source.data, 'image/png') ): source= "<img src=\"data:image/png;base64," + source.data['image/png'] + "\" alt='Image Alt Text'>"
  elif ( hasattr(source.data, 'text/plain') ): source = '' if re.search("<Figure", source.data['text/plain']) else source.data['text/plain'];
  # print(source)
  for lbl in flags:
    source = source.replace(lbl+'\r\n', "")
    source = source.replace(lbl+'\n', "") # Strip the Flag
    if (lbl == '#collapse_output_open'): source = makeDetails( source, True )
    if (lbl == '#collapse_output'): source = makeDetails( source, False )
    if (lbl == '#hide_output'): source = ''
    if (lbl == '#hide '): source = ''
  return source
  #output_type == 'stream' ==> text
  #output_type == 'display_data' ==> data{'application/javascript' or 'text/html' or 'execute_result'}

In [14]:
#export
# https://apps.timwhitlock.info/unicode/inspect/hex/1F633 Use: Dec => Code
def replaceEmojis(text):
    text = text.replace('🙂', '&#1F642')
    text = text.replace('😳', '&#128563')
    text = text.replace('\u2003', '&#8195')
    text = text.replace('👷', '&#1F477')
    # text = text.replace('🧡', 'hi') # &#1F9E1
    return text

In [15]:
#export
# Must be in directory of ipynb you want to convert to html.
def publish(fname='index'): 
  sys.path.pop()
  sys.path.append('workspaces/3Diot.github.io/docs/')
  %cd ../
  from blog import core 
  %cd notebooks 
  nb = read_nb(fname+'.ipynb')
  meta_jekyll = get_metadata(nb['cells'])
  outp = convertNb(nb)
  outp = ''.join( [''.join(c) for c in outp] ) 
  resp = ''.join( [''.join(c) for c in outp] )
  resp = replaceEmojis(resp)
  template = open("../docs/_templates/"+meta_jekyll['template']+".html", "r").read() 
  template = template.replace('{{tab}}', meta_jekyll['tab']) 
  template = template.replace('{{title}}', meta_jekyll['title']) 
  template = template.replace('{{filename}}', fname+'.ipynb') 
  txt = template.replace('{{content}}', resp ) 
  f = open("../docs/"+meta_jekyll['filename'].lower().replace(' ', '_')+".html", "w") # , encoding='utf-8'
  f.write(txt)
  f.close()
  return txt
  # return "<p>Hello, Worrrld!</p>"

In [17]:
from IPython. display import clear_output
publish('index')
publish('002_Parcels')
publish('003_TokenGating')
publish('004_Lore')
publish('006_Tutorials')
publish('007_Monetize')
publish('008_Inspirational')
publish('009_Ideas')
clear_output()

In [ ]:
nb = read_nb('index.ipynb')
outp = convertNb(nb)

In [ ]:
#export
def runit(fname='index.ipynb'): 
  nb = read_nb(fname)
  meta_jekyll = get_metadata(nb['cells'])
  outp = convertNb(nb)
  txt = ''.join( [''.join(c) for c in outp] )
  return txt

In [ ]:
pip install flask

In [ ]:
# sys.path.append('workspaces/3Diot.github.io/blog/')

In [ ]:
from flask import Flask
import re
app = Flask(__name__)

@app.route("/")
def homepage():  
  return publish('index') 
    
@app.route("/parcels")
def parcels():  
  return publish('002_Parcels') 

@app.route("/tokengating")
def progress():  
  return publish('003_TokenGating') 

@app.route("/lore")
def scarcity():  
  return publish('004_Lore') 

@app.route("/tutorials")
def technology():  
  return publish('006_Tutorials') 

@app.route("/monetize")
def tutorials():  
  return publish('007_Monetize') 


@app.route("/legal")
def legal():  
  return publish('999_Legal') 
  

app.run()

ModuleNotFoundError: No module named 'flask'

In [ ]:
t = runit(fname='002_Parcels.ipynb')

In [ ]:
import IPython
IPython.display.HTML(t)

In [ ]:
ls

000_core.ipynb        001c_Example.ipynb  013_CV_Misc.ipynb  index.ipynb
001a_core_test.ipynb  002_Parcels.ipynb   _devserver.ipynb
001b_core_test.ipynb  003_Scarcity.ipynb  cv_map.geojson


In [ ]:
sys.path.append('workspaces/3Diot.github.io/blog/')

In [ ]:
sys.path